In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST


In [3]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activations):
        super(FeedForwardNN, self).__init__()
        assert len(hidden_sizes) == len(activations), "Number of hidden layers should match number of activation functions"
        layers = []
        prev_size = input_size
        for i in range(len(hidden_sizes)):
            layers.append(nn.Linear(prev_size, hidden_sizes[i]))
            layers.append(activations[i])
            prev_size = hidden_sizes[i]
        layers.append(nn.Linear(prev_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [4]:
input_size = 28 * 28
hidden_sizes = [256, 128]
output_size = 10
activations = [nn.ReLU(), nn.ReLU()]

model = FeedForwardNN(input_size, hidden_sizes, output_size, activations)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [5]:
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


100%|██████████| 9912422/9912422 [00:00<00:00, 109767666.80it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 33407527.25it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31393061.26it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3417748.25it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.view(images.size(0), -1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.3513
Epoch [2/10], Loss: 0.0960
Epoch [3/10], Loss: 0.3057
Epoch [4/10], Loss: 0.1599
Epoch [5/10], Loss: 0.0525
Epoch [6/10], Loss: 0.1096
Epoch [7/10], Loss: 0.0012
Epoch [8/10], Loss: 0.0058
Epoch [9/10], Loss: 0.0403
Epoch [10/10], Loss: 0.0109


In [7]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on test images: {:.2f}%'.format(100 * correct / total))






Accuracy on test images: 97.72%
